In [156]:
from pathlib import Path
import subprocess
"""
Needs samtools, bowtie2, cutadapt, fastqc in the PATH and this is in python3.7
"""

path = "Raw_reads/"

# This can be any SCDS fastq raw read files
fastq_file = "SCDSPcelA-V1N-1_S75_L008_R1_001.fastq"

# Align index is default hsa_miRNA_mature_index, but can be hsa_miRNA_hairpin_index, hg19/hg19
align_index = "hsa_miRNA_mature_index"

### Used fastqc to check on the fastq raw read file 

In [157]:
report = subprocess.check_output(["fastqc", path + fastq_file])
print(report.decode("utf-8"))

Analysis complete for SCDSPcelA-V1N-1_S75_L008_R1_001.fastq



### Use the link below to see example of the report
[The QC of the fastq file](./Raw_reads/SCDSPcelA-V1N-1_S75_L008_R1_001_fastqc.html)

### Apparently adaptors are needed to be trimmed

In [155]:
# This step trimmed the NEB adapters
report = subprocess.check_output(['cutadapt',
                         '-f',
                         'fastq',
                         '-a',
                         'AGATCGGAAGAGCACACGTCT',
                         '-g',
                         'GTTCAGAGTTCTACAGTCCGACGATC',
                         '-e',
                         '0.15',
                         '-O',
                         '10',
                         '-m',
                         '14',
                         path + fastq_file,
                         '-o',
                         path + "trimmed_" + fastq_file])
print(report.decode("utf-8"))

This is cutadapt 1.18 with Python 2.7.15
Command line parameters: -f fastq -a AGATCGGAAGAGCACACGTCT -g GTTCAGAGTTCTACAGTCCGACGATC -e 0.15 -O 10 -m 14 Raw_reads/SCDSPcelA-V1N-1_S75_L008_R1_001.fastq -o Raw_reads/trimmed_SCDSPcelA-V1N-1_S75_L008_R1_001.fastq
Processing reads on 1 core in single-end mode ...
Finished in 155.84 s (45 us/read; 1.33 M reads/minute).

=== Summary ===

Total reads processed:               3,451,121
Reads with adapters:                 3,420,088 (99.1%)
Reads that were too short:              26,045 (0.8%)
Reads written (passing filters):     3,425,076 (99.2%)

Total basepairs processed:   172,556,050 bp
Total written (filtered):     92,852,383 bp (53.8%)

=== Adapter 1 ===

Sequence: AGATCGGAAGAGCACACGTCT; Type: regular 3'; Length: 21; Trimmed: 3418965 times.

No. of allowed errors:
0-5 bp: 0; 6-12 bp: 1; 13-19 bp: 2; 20-21 bp: 3

Bases preceding removed adapters:
  A: 15.4%
  C: 22.8%
  G: 10.4%
  T: 51.3%
  none/other: 0.1%

Overview of removed sequences
len

In [158]:
# Take another look with fastqc
trimmed_fastq_file = "trimmed_" + fastq_file
report = subprocess.check_output(["fastqc", path + trimmed_fastq_file])
print(report.decode("utf-8"))

Analysis complete for trimmed_SCDSPcelA-V1N-1_S75_L008_R1_001.fastq



### Use the link below to see example of the report
[The QC of the trimmed_fastq file](./Raw_reads/trimmed_SCDSPcelA-V1N-1_S75_L008_R1_001_fastqc.html)

In [159]:
# Make a miRNA hairpin and mature index file with Bowtie2 
hsa_miRNA_hairpin_index = Path(path + "hsa_miRNA_hairpin.fa")
if not hsa_miRNA_hairpin_index.is_file():
    res = []
    hsa = False
    with open(path + "hairpin.fa") as file:
        for line in file:
            if line.startswith(">hsa"):
                hsa = True
                res.append(line)
            elif line.startswith(">"):
                hsa = False
            else:
                if hsa:
                    res.append(line.replace("U", "T"))
    with open (path + "hsa_miRNA_hairpin.fa", "w") as file:
        for line in res:
            file.write(line)
    subprocess.check_output(["bowtie2-build", path + "hsa_miRNA_hairpin.fa", path+"hsa_miRNA_hairpin_index"])
    
hsa_miRNA_mature_index = Path(path + "hsa_miRNA_mature.fa")
if not hsa_miRNA_mature_index.is_file():
    res = []
    hsa = False
    with open(path + "mature.fa") as file:
        for line in file:
            if line.startswith(">hsa"):
                hsa = True
                res.append(line)
            elif line.startswith(">"):
                hsa = False
            else:
                if hsa:
                    res.append(line.replace("U", "T"))
    with open (path + "hsa_miRNA_mature.fa", "w") as file:
        for line in res:
            file.write(line)
    subprocess.check_output(["bowtie2-build", path + "hsa_miRNA_mature.fa", path+"hsa_miRNA_mature_index"])

In [160]:
# Extract sample name
def extract_sample_name(fastq_file):
    return fastq_file[4:fastq_file.index("-")]
sample_name = extract_sample_name(fastq_file)
print("Currently analyzing: "+sample_name)

Currently analyzing: PcelA


In [161]:
# Used bowtie2 to align the trimmed reads to align_index
# Method adapted from https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4652620/
subprocess.check_output(['bowtie2',
                         '-q',
                         '--phred33',
                         '-p',
                         '8',
                         '-D',
                         '20',
                         '--local',
                         '-R',
                         "3",
                         '-N',
                         '0',
                         '-L',
                         '8',
                         '-i',
                         "S,1,0.50",
                         '-x',
                         path + align_index,
                          path + trimmed_fastq_file,
                         '-S', path + "aln_" + sample_name +".sam"])
print("Done! The result is in " + path + "aln_" + sample_name +".sam")

Done! The result is in Raw_reads/aln_PcelA.sam


In [162]:
# Check the flatstat of the above aligned sam file
report = subprocess.check_output(["samtools", "flagstat",  path + "aln_" + sample_name +".sam"])
with open("Samtools_flagstat/" + sample_name + '_report.txt', "w") as outfile:
    for line in report.decode("utf-8"):
        outfile.write(line)
print(report.decode("utf-8"))

# Convert the sam file to bam file for downstream analysis
my_cmd = ["samtools", "view", "-S", "-b", path + "aln_" + sample_name +".sam"]
with open(path + "aln_" + sample_name +".bam", "w") as outfile:
    subprocess.call(my_cmd, stdout=outfile)

# Sort and index the bam file to make bai file
subprocess.check_output(["samtools", "sort", "-o", 
                         path + "sorted_" + "aln_" + sample_name +".bam", path + "aln_" + sample_name +".bam"])
subprocess.check_output(["samtools", "index",
                         path + "sorted_" + "aln_" + sample_name +".bam"])

# Redirect the samtools idxstats to a file in Raw_counts
from shlex import split
p1 = subprocess.Popen(split("samtools idxstats " + path + "sorted_" + "aln_" + sample_name +".bam"), stdout=subprocess.PIPE)
with open("Raw_counts/" + sample_name + "_raw_count.txt", "w") as outfile:
    subprocess.Popen(split("cut -f1,3"), stdin=p1.stdout, stdout=outfile)

3425076 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
1254190 + 0 mapped (36.62% : N/A)
0 + 0 paired in sequencing
0 + 0 read1
0 + 0 read2
0 + 0 properly paired (N/A : N/A)
0 + 0 with itself and mate mapped
0 + 0 singletons (N/A : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)

